In [ ]:
from PIL import Image
import os
from segments.utils import get_semantic_bitmap
import numpy as np
import matplotlib.pyplot as plt
from segments import SegmentsClient, SegmentsDataset
from datasets import Dataset
from datasets import concatenate_datasets
from huggingface_hub import login

In [ ]:

login('API_KEY')

In [ ]:
client = SegmentsClient('API_KEY')
release = client.get_release('hidonbush/text', 'paper cuttingv0.1')
dataset = SegmentsDataset(release, labelset='ground-truth', filter_by=['labeled', 'reviewed'])

Initializing dataset...
Preloading all samples. This may take a while...


100%|██████████| 1178/1178 [00:00<00:00, 3832.54it/s]

Initialized dataset with 1178 images.


In [5]:
def get_annotation(segdataset):
    annotaton_info={}
    for i in segdataset.release['dataset']['samples']:
        annotaton_info.update({i['name']:i['labels']['ground-truth']['attributes']['annotations']})
    return annotaton_info

In [6]:
def get_files(location):
    filenames=os.listdir(location)
    return filenames

In [7]:
def get_pictureset(files):
    pictureset=[]
    for i in files:
        if i[-23:]=='_label_ground-truth.png':
            pictureset.append({'name':i[:-23]+'.png','pixel_values':Image.open('/Users/zjy/Desktop/code/segments/hidonbush_text/paper cuttingv0.1/'+i[:-23]+'.png'),'label':Image.open('/Users/zjy/Desktop/code/segments/hidonbush_text/paper cuttingv0.1/'+i)})
    return pictureset


In [8]:
def create_dataset(annotaton_info,pictureset):
    resultlist_semantic_bitmap=[]
    resultlist_pixel_values=[]
    for i in pictureset:
        annotation=annotaton_info[i['name']]
        if annotation!=[]:
            semantic_bitmap=get_semantic_bitmap(i['label'],annotation)
            semantic_bitmap=Image.fromarray(semantic_bitmap.astype(np.int32)).convert('L')
            resultlist_semantic_bitmap.append(semantic_bitmap)
        else:
            height, width = i['label'].size
            semantic_bitmap=Image.fromarray(np.zeros((height, width), dtype=np.int32)).convert('L')
            resultlist_semantic_bitmap.append(semantic_bitmap)
        resultlist_pixel_values.append(i['pixel_values'].convert('RGB'))
    return {'pixel_values':resultlist_pixel_values,'label':resultlist_semantic_bitmap}

In [9]:
annotaton_info=get_annotation(dataset)

In [ ]:
files=get_files('Location')

In [11]:
pictureset=get_pictureset(files)

In [12]:
picdataset=create_dataset(annotaton_info=annotaton_info,pictureset=pictureset)

In [14]:
print(picdataset['pixel_values'][0])

<PIL.Image.Image image mode=RGB size=1169x1858 at 0x1070429E0>


In [15]:
hf_dataset01=Dataset.from_dict({'pixel_values':picdataset['pixel_values'][0:100],'labels':picdataset['label'][0:100]})

In [16]:
hf_dataset02=Dataset.from_dict({'pixel_values':picdataset['pixel_values'][100:300],'labels':picdataset['label'][100:300]})

In [17]:
hf_dataset03=Dataset.from_dict({'pixel_values':picdataset['pixel_values'][300:600],'labels':picdataset['label'][300:600]})

In [18]:
hf_dataset04=Dataset.from_dict({'pixel_values':picdataset['pixel_values'][600:900],'labels':picdataset['label'][600:900]})

In [19]:
hf_dataset05=Dataset.from_dict({'pixel_values':picdataset['pixel_values'][900:],'labels':picdataset['label'][900:]})

In [20]:
hf_dataset=concatenate_datasets([hf_dataset01,hf_dataset02,hf_dataset03,hf_dataset04,hf_dataset05])

In [21]:
hf_dataset_identifier = f"hidonbush/paper-cuttingv0.1"

hf_dataset.push_to_hub(hf_dataset_identifier)

Uploading the dataset shards:   0%|          | 0/2 [00:00<?, ?it/s]

Map:   0%|          | 0/589 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

Map:   0%|          | 0/589 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/352 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/hidonbush/paper-cuttingv0.1/commit/d692210e8d99b6a816db96eca77f7da8a954910e', commit_message='Upload dataset', commit_description='', oid='d692210e8d99b6a816db96eca77f7da8a954910e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/hidonbush/paper-cuttingv0.1', endpoint='https://huggingface.co', repo_type='dataset', repo_id='hidonbush/paper-cuttingv0.1'), pr_revision=None, pr_num=None)